In [1]:
import numpy as np
import pandas as pd

from scipy.stats import norm
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LassoCV, Lasso
from sklearn.preprocessing import StandardScaler    
from sklearn.model_selection import train_test_split

import ast
import random

CASE_VALUES = [0, 1, 5, 10, 25, 50, 75, 100, 200, 300, 400, 500, 750, 1000, 5000, 10000, 25000, 50000, 75000, 100000, 200000, 300000, 400000, 500000, 750000, 1000000]

CASES_PER_ROUND = [6, 5, 4, 3, 2, 1, 1, 1]

df = pd.read_csv(r"C:\\Users\\jjap2\\Poleto SCE\\Poleto-SCE\\dond_game_data.csv")

In [ ]:
df["Remaining Values"] = df["Remaining Values"].apply(ast.literal_eval)

case_values = set(CASE_VALUES)

def filter_remaining_values(row):
    return [value for value in row if value in case_values]

# Apply the filter
df["Remaining Values"] = df["Remaining Values"].apply(filter_remaining_values)

df = df[df["Remaining Values"].apply(lambda x: len(x) > 0)]

df = df[(df["Postgame"] != 1) & (df["Offer"] != 0)]

# Extract training features and target
exclude = ["Remaining Values", "Contestant Name", "Contestant Gender", "Contestant Race", "Amount Won", "Offer", "Game ID"]
X_train_data = df.drop(columns=exclude)
y_train_data = df["Offer"]

scaler = StandardScaler()

# Fit scaler on training data
## this one is fit transform because it's specifically on the training data
X_scaled_data = scaler.fit_transform(X_train_data)

# Train LassoCV model on the dataset
lasso = LassoCV(cv=5, random_state=42).fit(X_scaled_data, y_train_data)

In [3]:
def utility(x, alpha=0.01, beta=0.566):
    return (1 - np.exp(-alpha * x ** (1 - beta))) / alpha

In [4]:
def marginal_utility(x, alpha=0.01, beta=0.566):
    return np.exp(-alpha * x ** (1 - beta)) * x ** (-beta)

In [5]:
def simulate_decision(offer, board, alpha, beta, sigma, threshold=1.0):
    # Calculate the utility of the offer
    u_offer = utility(offer, alpha, beta)
    
    # Calculate the total utility of the board by summing up the utility of each case
    total_board_utility = sum(utility(val, alpha, beta) for val in board)
    
    # Calculate the average utility per remaining case on the board
    average_board_utility = total_board_utility / len(board) if len(board) > 0 else 0
    
    # Decision rule: accept the offer if its utility is higher than or equal to the average utility of the board
    decision = "Deal" if u_offer >= average_board_utility else "No Deal"
    
    # Calculate the probability of accepting the deal
    p_accept = u_offer / average_board_utility if average_board_utility > 0 else 0

    return {
        "Offer": offer,
        "Board Average": np.mean(board),
        "P(Deal)": p_accept,
        "Decision": decision
    }

In [6]:
def banker_offer_model(board, round_num, scaler, lasso_model, previous_offer=0):
    board_avg = np.mean(board)
    board_value = sum(board)
    board_balance = (max(board) - min(board)) / board_avg if board_avg > 0 else 0
    probability_of_big_value = np.mean(np.array(board) > 100000)
    
    # Generate binary vals for each case value based on the board state
    case_features = [1 if val in board else 0 for val in CASE_VALUES]
    
    features = {
        "Round": round_num,
        "Board Value": board_value,
        "Board Average": board_avg,
        "Board Balance": board_balance,
        "Previous Offer": previous_offer,
        "Offer Percent of Average": previous_offer / board_avg if board_avg > 0 else 0,
        "Deal": 0,
        "Postgame": 0,
        "Probability of Big Value": probability_of_big_value
    }

    for i, val in enumerate(CASE_VALUES):
        features[f"Case_{val}"] = case_features[i]

    # features match the training data columns
    features = {key: features[key] for key in X_train_data.columns if key in features}

    # Add missing features as zero - won't affect Lasso
    for column in X_train_data.columns:
        if column not in features:
            features[column] = 0

    # Create a DataFrame for the features and scale them
    # here we don't use fit_transform because there is already a scaler fit, basically calculates on a different scale than each other
    X = pd.DataFrame([features])
    X_scaled = scaler.transform(X)
    
    # Predict the offer using the Lasso model
    predicted_offer = lasso_model.predict(X_scaled)[0]
    
    # offer cant be negative
    return max(predicted_offer, 0)

In [7]:
def simulate_full_game(offer_function, alpha=0.01, beta=0.566, sigma=5.874, num_runs=1):
    results = []  # store results of each simulation run

    for _ in range(num_runs):
        values = CASE_VALUES.copy()
        np.random.shuffle(values)
        board = values.copy()
        player_case = board.pop(0)
        
        # initialize as no deal
        decision = "No Deal"
        final_case_value = player_case
        final_round = len(CASES_PER_ROUND)
        for round_num, num_to_open in enumerate(CASES_PER_ROUND, start=1):
            opened = np.random.choice(board, size=min(num_to_open, len(board)-1), replace=False)
            board = [val for val in board if val not in opened]
            offer = offer_function(board, round_num)  # Use the remaining cases
            result = simulate_decision(offer, board, alpha, beta, sigma)
            
            if result["Decision"] == "Deal":
                decision = "Deal"
                final_case_value = player_case
                final_round = round_num
                results.append({
                    "Decision": decision,
                    "Amount Won": round(offer, 2),  # Round the amount won to 2 decimal places
                    "Final Case Value": final_case_value,
                    "Final Round": final_round
                })
                break
            if len(board) <= 1:
                break
        if decision == "No Deal":  # If no deal was made during the whole game
            results.append({
                "Decision": "No Deal",
                "Amount Won": round(final_case_value, 2),  # Round the amount won to 2 decimal places
                "Final Case Value": final_case_value,
                "Final Round": final_round
            })
    return results

In [8]:
num_runs = 50  # loop x times
simulation_results = simulate_full_game(
    offer_function=lambda board, round_num: banker_offer_model(board, round_num, scaler, lasso),
    alpha=0.01, beta=0.566, sigma=5.874, num_runs=num_runs
)

for i, result in enumerate(simulation_results, start=1):
    print(f"Run {i}: {result}")

Run 1: {'Decision': 'Deal', 'Amount Won': 48000.08, 'Final Case Value': 5000, 'Final Round': 2}
Run 2: {'Decision': 'Deal', 'Amount Won': 28949.52, 'Final Case Value': 1, 'Final Round': 2}
Run 3: {'Decision': 'Deal', 'Amount Won': 26019.36, 'Final Case Value': 10, 'Final Round': 2}
Run 4: {'Decision': 'Deal', 'Amount Won': 24452.91, 'Final Case Value': 1000000, 'Final Round': 3}
Run 5: {'Decision': 'No Deal', 'Amount Won': 0, 'Final Case Value': 0, 'Final Round': 8}
Run 6: {'Decision': 'Deal', 'Amount Won': 27433.5, 'Final Case Value': 400000, 'Final Round': 2}
Run 7: {'Decision': 'Deal', 'Amount Won': 16939.5, 'Final Case Value': 300, 'Final Round': 5}
Run 8: {'Decision': 'Deal', 'Amount Won': 6160.25, 'Final Case Value': 0, 'Final Round': 2}
Run 9: {'Decision': 'No Deal', 'Amount Won': 5000, 'Final Case Value': 5000, 'Final Round': 8}
Run 10: {'Decision': 'No Deal', 'Amount Won': 100, 'Final Case Value': 100, 'Final Round': 8}
Run 11: {'Decision': 'Deal', 'Amount Won': 25567.7, 'Fina